In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [3]:
datapath = '/scratch/usr/shkifmjr/ORION10.L46.LIM2vp.CFCSF6.MOPS.JRA.XIOS2.5-EXP01/'\
           + 'eddy_tracking/example/'
meshpath = ['/scratch/usr/shkifmjr/NUSERDATA/ORION/10-data/meshmask/1_mesh_mask.nc']
data_in = sorted(glob(datapath + 'test_2018_grid_[TUV].nc'))

In [4]:
data = load_xorca_dataset(data_files=data_in, aux_files=meshpath, model_config='NEST',
           input_ds_chunks = {"time_counter": 73, "t": 73,
                              "z": 46, "deptht": 46, "depthu": 46, "depthv": 46, "depthw": 46,
                              "x": 100, "y": 100},
           target_ds_chunks = {"t": 73,
                               "z_c": 46, "z_l": 46,
                               "x_c": 100, "x_r": 100, "y_c": 100, "y_r": 100})

In [5]:
at, au = data['e1t'] * data['e2t'], data['e1u'] * data['e2u']
av, af = data['e1v'] * data['e2v'], data['e1f'] * data['e2f']
vt, vu, vv, vw = data['e3t'] * at, data['e3u'] * au, data['e3v'] * av, data['e3w'] * at

data = data.update({'at': at, 'au': au, 'av': av, 'af': af, 'vt': vt, 'vu': vu, 'vv': vv, 'vw': vw})
data = data.set_coords(['at', 'au', 'av', 'af', 'vt', 'vu', 'vv', 'vw'])

metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'], # Z distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'], # Areas
    ('X', 'Y', 'Z'): ['vt', 'vu', 'vv', 'vw'] # Volumes
}

In [7]:
# add bathymetry to data to have depth information
bathy = xr.open_dataset('/scratch/usr/shkifmjr/NUSERDATA/ORION/10-data/bathy_meter/'
                        + '1_bathy_meter__3.6.0_ORION10.L46_Kv1.0.0.nc')
data = data.update({'bathymetry': (['y_c', 'x_c'], bathy['Bathymetry'].data)})

In [8]:
grid = xgcm.Grid(data, metrics=metrics)

In [9]:
 # Calculate vorticity and Okubo-Weiss parameter and make sure the chunk sizes are as before.
data_OW = et.okuboweiss.calc(data.isel(z_c=9, z_l=9), grid,
                             'vozocrtx', 'vomecrty').chunk({'x_c': 3564, 'x_r': 3564,
                                                            'y_c': 629, 'y_r': 629})

In [10]:
# Merge the new variables `OW` and `vort` to the dataset `data`
data = xr.merge([data, data_OW], compat='override')

In [11]:
data

<xarray.Dataset>
Dimensions:     (t: 73, x_c: 3564, x_r: 3564, y_c: 629, y_r: 629, z_c: 46, z_l: 46)
Coordinates: (12/41)
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 9 10 ... 38 39 40 41 42 43 44 45 46
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 41.5 42.5 43.5 44.5 45.5
  * y_c         (y_c) int64 1 2 3 4 5 6 7 8 ... 622 623 624 625 626 627 628 629
  * y_r         (y_r) float64 1.5 2.5 3.5 4.5 5.5 ... 626.5 627.5 628.5 629.5
  * x_c         (x_c) int64 1 2 3 4 5 6 7 ... 3558 3559 3560 3561 3562 3563 3564
  * x_r         (x_r) float64 1.5 2.5 3.5 4.5 ... 3.562e+03 3.564e+03 3.564e+03
    ...          ...
    av          (y_r, x_c) float64 dask.array<chunksize=(100, 100), meta=np.ndarray>
    af          (y_r, x_r) float64 dask.array<chunksize=(100, 100), meta=np.ndarray>
    vt          (z_c, y_c, x_c) float64 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    vu          (z_c, y_c, x_r) float64 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    vv          (z_c, y_r, x_c) float64 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    vw          (z_l, y_c, x_c) float64 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
Data variables:
    sometauy    (t, y_r, x_c) float32 dask.array<chunksize=(73, 100, 100), meta=np.ndarray>
    vomecrty    (t, z_c, y_r, x_c) float32 dask.array<chunksize=(73, 46, 100, 100), meta=np.ndarray>
    sozotaux    (t, y_c, x_r) float32 dask.array<chunksize=(73, 100, 100), meta=np.ndarray>
    vozocrtx    (t, z_c, y_c, x_r) float32 dask.array<chunksize=(73, 46, 100, 100), meta=np.ndarray>
    vosaline    (t, z_c, y_c, x_c) float32 dask.array<chunksize=(73, 46, 100, 100), meta=np.ndarray>
    votemper    (t, z_c, y_c, x_c) float32 dask.array<chunksize=(73, 46, 100, 100), meta=np.ndarray>
    bathymetry  (y_c, x_c) float32 491.8 491.8 491.8 ... 4.056e+03 4.056e+03
    vort        (t, y_r, x_r) float32 dask.array<chunksize=(73, 629, 3564), meta=np.ndarray>
    OW          (t, y_r, x_r) float32 dask.array<chunksize=(73, 629, 3564), meta=np.ndarray>